<a href="https://colab.research.google.com/github/JulianaCarvajal/Spaceship_Titanic/blob/workOnPreprocess%2FBustamJos3/dataset_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#libraries
from zipfile import ZipFile
import pandas as pd
import numpy as np

# Working with the next cell [Online]
For now on, import cell from colab will have this code (**On local notebook, respective changes must be made in order to import .json token and datasets from kaggle**).

In [3]:
#call API
%pip install kaggle
#upload kaggle token .json
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bustamjos3","key":"a80d020fd1917115be9ce5ce749b71d9"}'}

In [4]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json
# Import data files from the Kaggle API
!kaggle competitions download -c spaceship-titanic
#unzip files
file_name = "spaceship-titanic.zip" #the file is the dataset exact name
with ZipFile("spaceship-titanic.zip", 'r') as zip:
  zip.extractall()
  print('Done')
# .zip is now not necessary
!rm *.zip

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 58.7MB/s]
Done


# End of working [Online]

# Working on Local

In [ ]:
#call API
%pip install kaggle
# Import data files from the Kaggle API
!kaggle competitions download -c spaceship-titanic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Unzip the data files
with ZipFile("spaceship-titanic.zip", 'r') as zip_ref:
    zip_ref.extractall()
    print('Done')
# .zip is now not necessary
!rm *.zip

# End of working on Local

In [87]:
# Read the data files into pandas dataframes
d_sTitanic = pd.read_csv("train.csv")
d_sTitanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Pipeline for work on preprocessing

In [88]:
#convert types of cols to the best possible dtype
d_sTitanic=d_sTitanic.convert_dtypes(infer_objects=True)

In [89]:
d_sTitanic.dtypes

PassengerId      string
HomePlanet       string
CryoSleep       boolean
Cabin            string
Destination      string
Age               Int64
VIP             boolean
RoomService       Int64
FoodCourt         Int64
ShoppingMall      Int64
Spa               Int64
VRDeck            Int64
Name             string
Transported     boolean
dtype: object

## Drop at 1st looking, not relevant columns:
* ```d_sTitanic['PassengerId']```
* ```d_sTitanic['Name']```

In [90]:
#drop those columns
d_sTitanic.drop(['PassengerId','Name'],axis=1,inplace=True)

In [91]:
# separation of X, y
y=d_sTitanic.iloc[:,-1].values

In [92]:
#also drop
d_sTitanic.drop(['Transported'],axis=1,inplace=True)

In [93]:
# take name cols which are categorical
listCategoric=[str(i) for i in (d_sTitanic.dtypes=='string').loc[(d_sTitanic.dtypes=='string')==True].index]

In [111]:
# take the categoric types only
#to handle oneHotEncoder, replace NaN values with 'wanted'
XCategoric=d_sTitanic[listCategoric].fillna('wanted').values
XCategoric

array([['Europa', 'B/0/P', 'TRAPPIST-1e'],
       ['Earth', 'F/0/S', 'TRAPPIST-1e'],
       ['Europa', 'A/0/S', 'TRAPPIST-1e'],
       ...,
       ['Earth', 'G/1500/S', 'TRAPPIST-1e'],
       ['Europa', 'E/608/S', '55 Cancri e'],
       ['Europa', 'E/608/S', 'TRAPPIST-1e']], dtype=object)

In [95]:
#numeric col names
listNumeric=[str(i) for i in (d_sTitanic.dtypes=='Int64').loc[(d_sTitanic.dtypes=='Int64')==True].index]

In [133]:
# take the numeric types only
XNumeric=d_sTitanic[listNumeric].fillna(-1).values
XNumeric

array([[39, 0, 0, 0, 0, 0],
       [24, 109, 9, 25, 549, 44],
       [58, 43, 3576, 0, 6715, 49],
       ...,
       [26, 0, 0, 1872, 1, 0],
       [32, 0, 1049, 0, 353, 3235],
       [44, 126, 4688, 0, 0, 12]], dtype=object)

## oneHotEncoding
Necessary to be done on previous 'cause working with the next methods require array-like data

In [97]:
from sklearn.preprocessing import OneHotEncoder

In [112]:
# instancing
oneHotEncoder=OneHotEncoder(handle_unknown='error',sparse=False)

In [113]:
#fitting with categorical X
oneHotEncoder.fit(XCategoric)

OneHotEncoder(sparse=False)

In [122]:
#array-like object with categories of the n-categoric cols
#'wanted' will be the nan values, placed at the end of each categoric col
oneHotEncoder.categories_

array(['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e', 'wanted'],
      dtype=object)

In [124]:
# tranformation to get convertion to OHE
convertedOHE=oneHotEncoder.transform(XCategoric)
convertedOHE

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.]])

In [125]:
#TO_CONSIDER!: categories for cabin column are a little infrequent***
oneHotEncoder.categories_[0].shape[0]+oneHotEncoder.categories_[1].shape[0]+oneHotEncoder.categories_[2].shape[0]

6569

## ```NaN``` Imputation

In [128]:
from sklearn.impute import KNNImputer

In [134]:
#instancing
kNNImputer=KNNImputer(n_neighbors=1,weights='distance')

In [137]:
#imputation
imputedNumeric=kNNImputer.fit_transform(XNumeric)
imputedNumeric

array([[3.900e+01, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
       [2.400e+01, 1.090e+02, 9.000e+00, 2.500e+01, 5.490e+02, 4.400e+01],
       [5.800e+01, 4.300e+01, 3.576e+03, 0.000e+00, 6.715e+03, 4.900e+01],
       ...,
       [2.600e+01, 0.000e+00, 0.000e+00, 1.872e+03, 1.000e+00, 0.000e+00],
       [3.200e+01, 0.000e+00, 1.049e+03, 0.000e+00, 3.530e+02, 3.235e+03],
       [4.400e+01, 1.260e+02, 4.688e+03, 0.000e+00, 0.000e+00, 1.200e+01]])

In [143]:
#let calculate the variance of imputedNumeric
np.var(imputedNumeric,axis=0)

array([2.23527767e+02, 4.36244410e+05, 2.54627368e+06, 3.57579568e+05,
       1.26675818e+06, 1.28610769e+06])

In [144]:
#let calculate the variance of convertedOHE
np.var(convertedOHE,axis=0)

array([0.24913614, 0.18504626, 0.16140252, ..., 0.08318324, 0.2174441 ,
       0.02049805])

### Now merge ```imputedNumeric``` and ```convertedOHE```

### Train-Test splitting

In [ ]:
# import train_test split function
from sklearn.model_selection import train_test_split

In [ ]:
# train validation split
X_trainST,X_valST,y_trainST,y_valST=train_test_split(X,y,test_size=0.33,random_state=42)

In [ ]:
# standardization is required 'cause many models work badly is data does'nt look like normally distributed data (see biblio)
from sklearn.preprocessing import StandardScaler

## Necessary
1. Drop cols with 0 importance
2. Make oneHotEncode to transform categorical to discrete. This is important 'cause preprocessing functions only work with numerical values!

In [ ]:
# scaling with train
standardS=StandardScaler().fit(X_trainST)

ValueError: ignored

# End of work on preprocessing